In [187]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate,train_test_split,KFold,LeaveOneOut,cross_val_score,cross_val_predict
from matplotlib import pyplot as plt
from sklearn.linear_model import Lasso
# from sklearn.linear_model import Ridge 
from sklearn.preprocessing import StandardScaler 
import seaborn as sns
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats


In [41]:
url = 'https://raw.githubusercontent.com/orangeczw/ML_team_Name/master/task1b_ow9d3sa/train.csv'
train = pd.read_csv(url, error_bad_lines=False)
y = train.iloc[:,1].copy()
X = train.iloc[:,2:].copy()
#See if there's NA value
X.isna().any()

x1    False
x2    False
x3    False
x4    False
x5    False
dtype: bool

In [109]:
X1 = X.apply(lambda x: np.square(x))
X1.columns = ['x6', 'x7', 'x8', 'x9', 'x10']
X2 = X.apply(lambda x: np.exp(x))
X2.columns = ['x11', 'x12', 'x13', 'x14', 'x15']
X3 = X.apply(lambda x: np.cos(x))
X3.columns = ['x16', 'x17', 'x18', 'x19', 'x20']
cons = np.empty(len(X))
cons.fill(1)
X4 = pd.DataFrame(cons)
X4.columns = ['x21']
X_all = X.join(X1).join(X2).join(X3).join(X4)

Xy_all = y.to_frame().join(X_all)
X_all.head(5)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21
0,0.02,0.05,-0.09,-0.43,-0.08,0.0004,0.0025,0.0081,0.1849,0.0064,...,1.051271,0.913931,0.650509,0.923116,0.999800,0.998750,0.995953,0.908966,0.996802,1.0
1,-0.13,0.11,-0.08,-0.29,-0.03,0.0169,0.0121,0.0064,0.0841,0.0009,...,1.116278,0.923116,0.748264,0.970446,0.991562,0.993956,0.996802,0.958244,0.999550,1.0
2,0.08,0.06,-0.07,-0.41,-0.03,0.0064,0.0036,0.0049,0.1681,0.0009,...,1.061837,0.932394,0.663650,0.970446,0.996802,0.998201,0.997551,0.917121,0.999550,1.0
3,0.02,-0.12,0.01,-0.43,-0.02,0.0004,0.0144,0.0001,0.1849,0.0004,...,0.886920,1.010050,0.650509,0.980199,0.999800,0.992809,0.999950,0.908966,0.999800,1.0
4,-0.14,-0.12,-0.08,-0.02,-0.08,0.0196,0.0144,0.0064,0.0004,0.0064,...,0.886920,0.923116,0.980199,0.923116,0.990216,0.992809,0.996802,0.999800,0.996802,1.0


### 1. With standardization

In [215]:
# scaling the inputs 
scaler = StandardScaler() 
scaled_X = scaler.fit_transform(X_all) 

penalty = list(np.linspace(0.05,0.2,100))

RMSE = []

for lmd in penalty:
    model = Lasso(alpha = lmd, fit_intercept = False)
    cv_results = cross_validate(model, scaled_X, y, cv=10,
                            n_jobs=-1,scoring=('neg_mean_squared_error'))
    result = np.sqrt(np.abs(cv_results["test_score"]).mean())
    RMSE.append(result)
    
# RMSE = pd.DataFrame(RMSE) 

# RMSE.to_csv(r'C:\Users\l\Desktop\task_1b.csv', index = False, header=False)
# np.savetxt("submission_1a.csv",np.array(RMSE))
# print(RMSE)

lmd = penalty[RMSE.index(min(RMSE))]
print(lmd)
model = Lasso(alpha = lmd, fit_intercept=False)
model.fit(scaled_X, y)

print(model.coef_)

0.2
[ 0.         -0.          0.          0.          0.         -0.01931393
 -0.01948427 -0.         -0.          0.          0.         -0.
  0.          0.27389037  0.          0.          0.          0.
  0.         -0.          0.        ]


#### 1.1 Add interception to linear_regression (skip here :)

In [216]:
# selected = (abs(model.coef_) > 0.01).tolist()
# x_selected = [i for i, x in enumerate(selected) if x]

# lasso_X = X_all.iloc[:,x_selected]
# linear_regression = LinearRegression(n_jobs=-1,fit_intercept = True)
# linear_regression.fit(lasso_X, y)
# print(linear_regression.coef_, linear_regression.intercept_)

# coefs = np.zeros(21)
# coefs[20] = linear_regression.intercept_
# coefs[x_selected]=linear_regression.coef_.tolist()

# print(coefs)

# coefs = pd.DataFrame(coefs) 
# coefs.to_csv(r'C:\Users\l\Desktop\task_1b.csv', index = False, header=False)

# # np.savetxt("submission_1b.csv",coefs)

#### 1.2 Without interception in linear_regression

In [218]:
selected = (abs(model.coef_) > 0.01).tolist()
x_selected = [i for i, x in enumerate(selected) if x]

lasso_X = X_all.iloc[:,x_selected]
linear_regression = LinearRegression(n_jobs=-1,fit_intercept = False)
linear_regression.fit(lasso_X, y)
# print(linear_regression.coef_)
coefs = np.zeros(21)
# print(linear_regression.coef_)
coefs[x_selected]=linear_regression.coef_.tolist()
print(coefs)

# coefs = pd.DataFrame(coefs) 
# coefs.to_csv(r'C:\Users\l\Desktop\task_1b.csv', index = False, header=False)

np.savetxt("submission_1b.csv",coefs)

[ 0.          0.          0.          0.          0.         -7.11607845
 -8.52928211  0.          0.          0.          0.          0.
  0.          1.76657279  0.          0.          0.          0.
  0.          0.          0.        ]


#### 1.3 Stat. of linear_regression (no interception)

In [209]:
# lasso_X1 = sm.add_constant(lasso_X)
model_reduced = sm.OLS(y, lasso_X)
final_model = model_reduced.fit()
print(final_model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     6.577
Date:                Wed, 11 Mar 2020   Prob (F-statistic):           9.01e-07
Time:                        19:24:52   Log-Likelihood:                -2102.9
No. Observations:                 700   AIC:                             4218.
Df Residuals:                     694   BIC:                             4245.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x3             1.4662      2.288      0.641      0.5

### 2. Without standardization (skip :)

In [152]:
# penalty = [0.0001,0.001,0.01,0.1,1,10,100,1000,10000]
penalty = [0,0.001,0.01,0.05,0.1,0.5,1]

RMSE = []

for lmd in penalty:
    model = Lasso(alpha = lmd, fit_intercept = False)
    cv_results = cross_validate(model, X_all, y, cv=10,
                            n_jobs=-1,scoring=('neg_mean_squared_error'))
    result = np.sqrt(np.abs(cv_results["test_score"]).mean())
    RMSE.append(result)
    
# RMSE = pd.DataFrame(RMSE) 

# RMSE.to_csv(r'C:\Users\l\Desktop\task_1b.csv', index = False, header=False)
# np.savetxt("submission_1a.csv",np.array(RMSE))
# print(RMSE)

lmd = penalty[RMSE.index(min(RMSE))]
print(lmd)
model = Lasso(alpha = lmd, fit_intercept=False)
model.fit(X_all, y)
print(model.coef_)

# lasso_X = X_all.iloc[:,13]
# linear_regression = LinearRegression(n_jobs=-1,fit_intercept = False)
# linear_regression.fit(lasso_X, y)
# print(linear_regression.coef_)
# coefs = np.zeros(21)
# coefs[13]=linear_regression.coef_[0]

0.05
[ 0.         -0.          0.          0.          0.         -0.
 -0.         -0.         -0.          0.          0.          0.
  0.          1.29630057  0.          0.          0.          0.
  0.          0.          0.        ]
